# Průvodce implementací trénování modelu

## Importy

In [1]:
# Ignoruje warningy dependencies
import warnings
warnings.filterwarnings('ignore')

# Python packages
import os
import sys

# DataScience packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Prida cestu pro custom moduly
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Vlastní package
from dataset_manager.dataset_manager import DatasetManager
from neural_network_lab.model_preset.logger import Logger

# Importuje nastaveni modelu s implementaci
from neural_network_lab.ann_classification import ModelNeuralNetwork

C:\python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Parametry datasetu

In [3]:
# Data Parameters
currency_pair = 'USD/JPY'
data_postfix = '12-16'
time_frame = '1H'

## Zpracování dat
1. Importuje data 
2. Vyčistí data od prázdných hodnot
3. Agreguje data na požadovanou hodnotu periody (default = 1H) 
4. Resetuje index
5. Uloží kopii do mezipaměti (slouží primárně k iterování)

In [4]:
# INITIALIZE DATASET
# ------------------------------------------------------------------------------
dm = DatasetManager(currency_pair, data_postfix).resample(time_frame)
dm.df.reset_index(drop=True, inplace=True)
dm.save_df_copy_into_memory()
dm.df.head()

Import dataset: usdjpy_12-16.csv
Clean data
Aggregate data on 1H candles


,open,high,low,close
0,76.948,76.991,76.923,76.984
1,76.985,77.002,76.926,76.948
2,76.947,76.947,76.894,76.921
3,76.920,76.924,76.896,76.906
4,76.903,76.914,76.898,76.907


## Inicializace modelu neuronové sítě
Vytvoří instanci model, která implementuje celkovou funkcionalitu modelu a pomocných metod
### Přepsání defaultních parametrů
I když je klasickým postupem definování parametrů uvnitř souboru modelu. Z důvodů hromadného učení desítek modelů s podobnými parametry během jedné iterace lze parametry přepsat i za běhu. Pokud chceme importovat existující model, je potřeba přepsat parametry odpovídající modelu.
#### model.predict_ma
- délka klouzavého průměru, který chceme predikovat
#### model.n_past
- parametr klouzavého okna
- model uvidí n_past period do minulosti
#### model.n_past
- parametr klouzavého okna
- model bude predikovat n_future period do budoucnosti

#### Parametry názvů modelu
- model.model_task
- model.model_postfix

In [8]:
# IMPORT NEURAL NETWORK
# ------------------------------------------------------------------------------
model = ModelNeuralNetwork(data_manager=dm)

# parameters
model.predict_ma: int = 40
model.n_past: int = 50
model.n_future: int = 10
model.model_task: str = 'classification'
model.model_postfix: str = 'test_spyder'

print(f'Model called {model.model_name} was initiated')

Initialize Model Builder
Initialize Model Evaluation
Initialize Model Strategies
Initialize Neural Network Setup
Model called classification_usdjpy_12-16_MA40_past50_fut10_test_spyder was initiated


## Logger
Uchovává základní informace o parametrech modelu a výsledky jeho evaluace. Soubor je vytvořen ve složce modelu.

In [11]:
# LOGGER
# ------------------------------------------------------------------------------
logger = Logger()
logger.set_model(model)
logger.set_data_manager(dm)

## Základní vstupy
- Sada klouzavých průměrů
- RSI indikátor
- Stochastic oscilátor
- uložení indikátorů pro další operace

In [5]:
# Get clean df from memory
dm.restore_df()

# INDICATORS
# ------------------------------------------------------------------------------
dm.ewma(model.predict_ma)
if model.predict_ma != 15:
    dm.ewma(15)
if model.predict_ma != 20:
    dm.ewma(20)
if model.predict_ma != 30:
    dm.ewma(30)
if model.predict_ma != 40:
    dm.ewma(40)
if model.predict_ma != 60:
    dm.ewma(60)
dm.rsi_indicator(25)
dm.stochastic_oscilator(25, 3, 3)
dm.set_indicators(target=model.model_task)

# First 5 rows
dm.df.head()

## Odvozené vstupy z EMA
- zlogaritmovaná lineární regrese za minulé období o délce n_future
- sada rozdílů EMA za 1 období
- sada zlogaritmovaných regresí za 1 období

In [7]:
# Derived Quantities
dm.df['past_price_regression'] = dm.df[dm.mean_indicators[0]] / dm.df[dm.mean_indicators[0]].shift(
    model.n_future)
dm.df['past_log_regression'] = np.log(dm.df['past_price_regression'])
for mean_average in dm.mean_indicators:
    dm.df['mean_diff_{}'.format(mean_average[-2:])] = dm.df[mean_average] - dm.df[
        mean_average].shift(1)
    dm.df['mean_ret_{}'.format(mean_average[-2:])] = np.log(
        dm.df[mean_average] / dm.df[mean_average].shift(1))

TypeError: 'set' object does not support indexing

## Výpočet klasifikačních tříd
- třída 1: klouzavý průměr za budoucích n_future rostl
- třída 0: klouzavý průměr za budoucích n_future klesal

In [14]:
# CLASSIFICATION VALUES
dm.df['future_price_regression'] = dm.df[dm.mean_indicators[0]].shift(-model.n_future) / dm.df[
    dm.mean_indicators[0]]
dm.df[model.model_task] = np.where(dm.df['future_price_regression'] > 1, 1, 0)

IndexError: list index out of range

## Odstranění nepotřebných dat
- odstraní se nepotřebné sloupce
- odstraní se těsné okraje datasetu, na kterých se nachází prázdné hodnoty ve sloupcích indikátorů (indikátor neměl dostatek dat, proto nemohl spočítat hodnotu)
- resetování indexu a uložení zbývajících idikátorů


In [ ]:
# Drop unnecessary values
dm.df.drop(['low', 'high', 'open'], axis=1, inplace=True)
dm.df.drop(['%d', 'past_price_regression', 'future_price_regression'], axis=1, inplace=True)
# for training without mean averages
# dm.df.drop(model.mean_indicators, axis=1, inplace=True)
dm.df = dm.df.iloc[30:-5]
dm.df.reset_index(drop=True, inplace=True)
dm.set_indicators(target=model.model_task)
df = dm.df
df.head()

## Rozdělení datasetu na Test / Train
- rozdělí dle parametru v třídě modelu, defaultně je velikost **test datasetu 20 %** z celkových dat (model.test_size = 0.2)
- validační dataset se vytvoří automaticky při začátku trénování, defaultně je **velikost 15 % z trénovacího setu** (model.val_size = 0.15)

In [ ]:
# Test/Train split
df_train, df_test, df_test_close = dm.test_train_split(model)

## Normalizace dat
Vstupy se normalizují pomocí **Z-Score** metody.

In [ ]:
# NORMALIZATION
# ------------------------------------------------------------------------------
scaler = StandardScaler()
scaled_df_train = scaler.fit_transform(df_train[dm.mean_indicators + dm.indicators])
scaled_df_test = scaler.transform(df_test[dm.mean_indicators + dm.indicators])

## Vytvoření vstupních vektorů - plovoucí okna
Pomocí implementované metody se proiterují datasety a vytvoří se vstupní trénovací a testovací vektory.

In [ ]:
x_train, y_train = model.create_train_vectors(df_train, scaled_df_train)
x_test, y_test, y_test_price = model.create_test_vectors(df_test, scaled_df_test, df_test_close)

## Trénování neuronové sítě
1. Model **dosadí hyperparametry **
2. Model se **skompiluje**
3. Model definuje chování během trénování = **Callbacks**
    - dynamické trénování
    - dynamický learning rate
    - model checkpoint
4. Model **spustí proces učení**
5. **Váhy se ukládají** během učení, pokud model dosáhne nového lepšího výsledku
6. Model **ukončí své trénování**, pokud se nadále nezlepšuje
7. Model uloží své **trénovací, validační skóre**

#### Poznámka:
parametr verbose - zobrazí proces trénování

In [ ]:
# TRAIN NETWORK
# ------------------------------------------------------------------------------
trained_model, training_history = model.train_network(x_train, y_train, verbose=0)
# Plot Training Progress of Error
model.plot_training_loss()

#### Chyba během trénování

In [ ]:
# Plot Training Progress of Error
model.plot_training_loss()

#### Přesnost během trénování

In [ ]:
# Plot Training Progress of Accuracy
model.plot_training_metric()

# Klasifikace hodnot
- **načte váhy** modelu uložené během trénování
- **klasifikuje třídy** na testovacím / trénovacím setu

In [ ]:
# MAKE PREDICTION
# ------------------------------------------------------------------------------
# Load Best Model
classifier = model.load_network()
# Make Predictions
predictions_train = classifier.predict(x_train)
predictions_test = classifier.predict(x_test)
# Set values for evaluation
actual_train = y_train
actual_test = y_test

## Vytvoření setů pro evaluaci
- **Sloupce**: Aktuální hodnoty + Predikované hodnoty + Uzavírací cena

In [ ]:
# CREATE SETS FOR EVALUATION
# Columns: Actual, Prediction, Close Price
# ------------------------------------------------------------------------------
# TRAIN Evaluation Set
df_train_eval = model.create_train_eval_set(actual_train, predictions_train)
# VALIDATION Evaluation Set
df_val_eval = model.create_val_eval_set(actual_train, predictions_train)
# TEST Evaluation Set
df_test_eval = model.create_test_eval_set(actual_test, predictions_test, y_test_price)

## Přesnost na testovacím setu

In [ ]:
# ACCURACY EVALUATION
# ------------------------------------------------------------------------------
model.test_score = model.calc_acc(df_train_eval.copy(), origin=0.5, actual_col='actual', prediction_col='prediction')